## Compumedics Profusion PSG Report Extractor
A common codeblock for RTF and DOCX files.

In [ ]:
! pip install pandas python-docx striprtf tqdm


In [153]:
import os
import re
import pandas as pd
from docx import Document
from striprtf.striprtf import rtf_to_text
from tqdm import tqdm

In [ ]:
folder_path = "<FOLDER PATH HERE>"

In [ ]:
# Function to extract text from a DOCX file
def docx_to_text_with_tables(file_path):
    try:
        # Load the DOCX file
        doc = Document(file_path)
        
        # Extract all the text
        full_text = []
        
        # Process paragraphs
        for para in doc.paragraphs:
            full_text.append(para.text)
        
        # Process tables
        for table in doc.tables:
            for row in table.rows:
                row_text = ' | '.join(cell.text.strip() for cell in row.cells)
                full_text.append(row_text)
        
        # Join all the paragraphs and table rows into a single string
        return '\n'.join(full_text)
    except Exception as e:
        print(f"Error processing DOCX file {file_path}: {e}")
        return None

# Function to preprocess DOCX content
def preprocess_docx_content(content):
    # Split content into lines
    lines = content.split('\n')
    processed_lines = []
    for line in lines:
        # Replace the second '|' in every line with a newline
        parts = line.split('|', 2)  # Split into at most three parts
        if len(parts) == 3:
            line = parts[0] + '|' + parts[1] + '\n' + parts[2]
        # Remove trailing '|' at the end of the line
        if line.endswith('|'):
            line = line[:-1]
        processed_lines.append(line)
    return '\n'.join(processed_lines)

# Function to preprocess RTF content
def preprocess_rtf_content(content):
    # Split content into lines
    lines = content.split('\n')
    processed_lines = []
    for line in lines:
        # Replace the second '|' in every line with a newline
        parts = line.split('|', 2)  # Split into at most three parts
        if len(parts) == 3:
            line = parts[0] + '|' + parts[1] + '\n' + parts[2]
        # Remove trailing '|' at the end of the line
        if line.endswith('|'):
            line = line[:-1]
        processed_lines.append(line)
    return '\n'.join(processed_lines)

# Function to extract information after a keyword
def extract_information(preprocessed_text, keyword):
    # Escape special characters in the keyword for regex
    keyword_escaped = re.escape(keyword.strip())

    # Extract the value after the keyword using the | delimiter or end of line
    pattern = re.compile(rf'{keyword_escaped}\s*\|\s*([^\|\n]+)')
    match = pattern.search(preprocessed_text)

    if match:
        return match.group(1).strip()
    else:
        # Try again with optional spaces around the keyword
        pattern_with_spaces = re.compile(rf'\s*{keyword_escaped}\s*\|\s*([^\|\n]+)')
        match_with_spaces = pattern_with_spaces.search(preprocessed_text)
        if match_with_spaces:
            return match_with_spaces.group(1).strip()
        return None

# Function to extract plain text between keywords
def extract_plaintext(preprocessed_text, keyword, end_keywords=None):
    # Escape special characters in the keyword for regex
    keyword_escaped = re.escape(keyword.strip())

    # Extract the value between the keyword and the end keywords
    if end_keywords:
        end_keywords_escaped = '|'.join(re.escape(end_kw.strip()) for end_kw in end_keywords)
        pattern = re.compile(rf'{keyword_escaped}\s*(.*?)\s*(?:{end_keywords_escaped})', re.DOTALL)
    else:
        pattern = re.compile(rf'{keyword_escaped}\s*([^\|\n]+)')
    
    match = pattern.search(preprocessed_text)

    if match:
        return match.group(1).strip()
    else:
        #print(f"No match found for keyword '{keyword}'.")  # Debugging statement for no match
        return None

# Function to get the next line after a keyword
def get_next_line_after_keyword(preprocessed_text, keyword):
    # Escape special characters in the keyword for regex
    keyword_escaped = re.escape(keyword.strip())

    # Split the content into lines
    lines = preprocessed_text.split('\n')

    # Iterate over lines to find the keyword
    for i, line in enumerate(lines):
        if re.search(keyword_escaped, line):
            # Return the next line if it exists
            if i + 1 < len(lines):
                return lines[i + 1].strip()
            else:
                return None

    # Return None if the keyword is not found or there is no next line
    return None

# Function to get text until end of line after a keyword
def get_text_after_keyword_until_eol(content, keyword):
    # Escape special characters in the keyword for regex
    keyword_escaped = re.escape(keyword.strip())
    
    # Split the content into lines
    lines = content.split('\n')
    
    # Iterate over lines to find the keyword
    for line in lines:
        if re.search(keyword_escaped, line):
            # Extract the text after the keyword until the end of the line
            pattern = re.compile(rf'{keyword_escaped}(.*)')
            match = pattern.search(line)
            if match:
                result = match.group(1).strip()
                # Strip leading or trailing '|' characters
                result = result.strip('|')
                # Split by '|' and return as list of columns
                columns = result.split('|')
                return [col.strip() for col in columns]
    
    # Return None if the keyword is not found
    return None

# Function to get text after the 2nd occurrence of a keyword until end of line
def get_text_after_2nd_keyword_until_eol(content, keyword):
    # Escape special characters in the keyword for regex
    keyword_escaped = re.escape(keyword.strip())
    
    # Split the content into lines
    lines = content.split('\n')
    
    # Iterate over lines to find the keyword
    match_count = 0
    for line in lines:
        if re.search(keyword_escaped, line):
            match_count += 1
            if match_count == 2:
                # Extract the text after the keyword until the end of the line
                pattern = re.compile(rf'{keyword_escaped}(.*)')
                match = pattern.search(line)
                if match:
                    result = match.group(1).strip()
                    # Strip leading or trailing '|' characters
                    result = result.strip('|')
                    # Split by '|' and return as list of columns
                    columns = result.split('|')
                    return [col.strip() for col in columns]
    
    # Return None if the keyword is not found or there's no 2nd occurrence
    return None

def get_all_docx_files(folder_path):
    docx_files = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(('.docx', '.DOCX')):
                docx_files.append(os.path.join(root, file))
    return docx_files

def get_all_rtf_files(folder_path):
    docx_files = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(('.rtf', '.RTF')):
                docx_files.append(os.path.join(root, file))
    return docx_files

# Function to process both DOCX and RTF files
def process_files(folder_path):
    # List all DOCX and RTF files in the folder
    docx_files = get_all_docx_files(folder_path)
    rtf_files = get_all_rtf_files(folder_path)
    #docx_files = [f for f in os.listdir(folder_path) if f.endswith(('.docx', '.DOCX'))]
    #rtf_files = [f for f in os.listdir(folder_path) if f.endswith(('.rtf', '.RTF'))]
    print(f"DOCX files found: {len(docx_files)}")  # Debugging statement to check the number of DOCX files found
    print(f"RTF files found: {len(rtf_files)}")  # Debugging statement to check the number of RTF files found

    # Initialize a list to store the extracted data
    data = []

    # Keywords to extract

    # CUSTOMIZE THIS LIST FOR YOUR OWN REPORT. 
    # keywords will return the characters between the keyword and "|"
    # keywords_nextline will return next line after the keyword.
    # keywords_eol will return all characters until end of that line after the keyword.
    # keywords_eol_2nd will return all characters until end of that line after the 2nd time the keyword was found.
    keywords = ["AD-SOYAD", "TC KİMLİK NO", "DOĞUM TARİHİ", "DOĞUM YERİ", "CİNSİYET -YAŞ", "KAYIT TARİHİ", 
                "Kilo (kg)", "Boy (cm)", "Boyun Çev.(cm)", "Bel/Kalça(cm)", 
                "Işık Kapanış Zamanı:", "Toplam Kayıt Süresi (dk):", "Uyanış Sayısı:", "Uykuya Dalış Süresi (dk):", 
                "Uykuya Dalış Sonrası Uyanışların\nSüresi (dk):", "Toplam Uyku Süresi (dk):", "Uyku Etkinliği (%):", 
                "İlk REM Evresine Giriş Süresi (dk):", "UYANIK(sleep time)", "SaO2 Desat. İndex(ODI)",
                "Tüm Uykuda Ortalama Nabız Sayısı", "REM Evresinde Ortalama Nabız Sayısı","NREM Evresinde Ortalama Nabız Sayısı",
                "Bradikardik Dönemlerin Sayısı","Taşikardik Dönemlerin Sayısı"]
    keywords_nextline = ["Toplam Arousal İndeksi"]
    keywords_eol = ["NREM 1", "NREM 2", "NREM 3", 
                    "Back(Sırt Üstü)", "Prone(Yüz Üstü)", "Left(Sol)", "Right(Sağ)", "NonSupine",
                    "AHI","Ortalama Süre (sn)",
                    "Apneler + Hipopneler","RERALAR","Apnelerin Süresi (dk)",
                    "Hipopnelerin Süresi (dk)","Apne + Hipopneler (dk)","AHI (A+H/saat)",
                    "Respiratuar Arousal İndeks (RDI)","Respiratuar Arousallar","Respiratuar Disturbance İndeks(RDI)",
                    "Periyodik Ekstremite Hareketleri Sayısı","Ekstremite Hareketleri ile ilgili Arousalların Sayısı",
                    "PLM İndeksi","Periyodik Ekstremite  Hareketleri Arousal İndeksi","ARO RES","ARO SPONT","ARO Limb","ARO PLM"]
    keywords_eol_2nd = ["REM ","Apneler","Hipopneler"]

    # Process each DOCX file
    for docx_file in tqdm(docx_files, desc="Processing files", unit="file"):
        
        file_path = os.path.join(folder_path, docx_file)
        #print(f"Processing file: {docx_file}")  # Debugging statement to check file processing
        row = {"File": docx_file}

        content = docx_to_text_with_tables(file_path)
        if not content:
            continue

        preprocessed_text = preprocess_docx_content(content)

        for keyword in keywords:
            value = extract_information(preprocessed_text, keyword)
            #print(f"{keyword}: {value}")  # Debugging statement to check extracted values
            row[keyword] = value

        for keyword in keywords_nextline:
            value = get_next_line_after_keyword(preprocessed_text, keyword)
            if value is not None:
                value = value.replace('|', '')
                #print(f"{keyword}2: {value}")  # Debugging statement to check extracted values
                row[keyword+"_nl"] = value

        for keyword in keywords_eol:
            value_array = get_text_after_keyword_until_eol(content, keyword)
            #print(f"{keyword}_eol: {value_array}")  # Debugging statement to check extracted values
            i = 0
            if value_array is not None:
                for value in value_array:
                    row[keyword+"_"+str(i)] = value
                    #print(f"{keyword}_{i}: {value_array[i]}")  # Debugging statement to check extracted values
                    if i == 4: break
                    i = i + 1

        for keyword in keywords_eol_2nd:
            value_array = get_text_after_2nd_keyword_until_eol(content, keyword)
            #print(f"{keyword}_eol: {value_array}")  # Debugging statement to check extracted values
            i = 0
            if value_array is not None:
                for value in value_array:
                    row[keyword+"_"+str(i)] = value
                    #print(f"{keyword}_{i}: {value_array[i]}")  # Debugging statement to check extracted values
                    if i == 4: break
                    i = i + 1

        # Extract the text between "Tanı:" and "Prof", "Uz", or "Doç"
        diagnosis = extract_plaintext(preprocessed_text, "Tanı", ["Prof", "Uz", "Doç"])
        #print(f"Diagnosis: {diagnosis}")  # Debugging statement to check extracted diagnosis
        row["Tanı"] = diagnosis

        data.append(row)

    # Process each RTF file
    for rtf_file in tqdm(rtf_files, desc="Processing files", unit="file"):
        file_path = os.path.join(folder_path, rtf_file)
        #print(f"Processing file: {rtf_file}")  # Debugging statement to check file processing
        row = {"File": rtf_file}

        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                rtf_content = file.read()
        except UnicodeDecodeError:
            with open(file_path, 'r', encoding='latin1') as file:
                rtf_content = file.read()

        plain_text = rtf_to_text(rtf_content)
        preprocessed_text = preprocess_rtf_content(plain_text)

        for keyword in keywords:
            value = extract_information(preprocessed_text, keyword)
            #print(f"{keyword}: {value}")  # Debugging statement to check extracted values
            row[keyword] = value

        for keyword in keywords_nextline:
            value = get_next_line_after_keyword(preprocessed_text, keyword)
            if value is not None:
                value = value.replace('|', '')
                #print(f"{keyword}_nl: {value}")  # Debugging statement to check extracted values
                row[keyword+"_nl"] = value

        for keyword in keywords_eol:
            value_array = get_text_after_keyword_until_eol(plain_text, keyword)
            #print(f"{keyword}_eol: {value_array}")  # Debugging statement to check extracted values
            i = 0
            if value_array is not None:
                for value in value_array:
                    row[keyword+"_"+str(i)] = value
                    #print(f"{keyword}_{i}: {value_array[i]}")  # Debugging statement to check extracted values
                    if i == 4: break
                    i = i + 1

        for keyword in keywords_eol_2nd:
            value_array = get_text_after_2nd_keyword_until_eol(plain_text, keyword)
            #print(f"{keyword}_eol: {value_array}")  # Debugging statement to check extracted values
            i = 0
            if value_array is not None:
                for value in value_array:
                    row[keyword+"_"+str(i)] = value
                    #print(f"{keyword}_{i}: {value_array[i]}")  # Debugging statement to check extracted values
                    if i == 4: break
                    i = i + 1

        # Extract the text between "Tanı:" and "Prof", "Uz", or "Doç"
        diagnosis = extract_plaintext(preprocessed_text, "Tanı", ["Prof", "Uz", "Doç"])
        #print(f"Diagnosis: {diagnosis}")  # Debugging statement to check extracted diagnosis
        row["Tanı"] = diagnosis

        data.append(row)

    # Create a DataFrame from the extracted data
    df = pd.DataFrame(data)
    return df

# Process the files and create a DataFrame
df = process_files(folder_path)

df_yedek = df

# Print the DataFrame
display(df)

### Data cleanup

In [156]:
def strip_units_from_all_columns(df, units):
    """
    Strips specified units from the values in all columns of a DataFrame.

    Parameters:
    df (pd.DataFrame): The DataFrame to process.
    units (list): A list of unit strings to strip from the values.

    Returns:
    pd.DataFrame: The DataFrame with the processed columns.
    """
    # Create a regular expression pattern to match and remove the units
    unit_pattern = '|'.join(map(re.escape, units))
    
    # Iterate through each column in the DataFrame
    for column in df.columns:
        # Apply the unit stripping and convert to numeric
        df[column] = df[column].astype(str).str.replace(unit_pattern, '', regex=True).str.strip()
        
    
    return df

# Strip units 'dk' and '%' from all columns
units_to_strip = ['dk', '%']
df = strip_units_from_all_columns(df, units_to_strip)


In [157]:
def process_tani(tani):
    # Replace tab characters with spaces
    tani = tani.replace('\t', ' ')
    
    # Split by newlines
    parts = re.split(r'\n', tani)
    
    # Clean each part
    cleaned_parts = []
    for part in parts:
        # Remove leading numbers, periods, or dashes within the first 5 characters
        part = re.sub(r'^[\s\d.-:]{1,10}', '', part.strip())
        # Remove the last period
        part = part.rstrip('.')
        # Append to cleaned parts if it's not empty
        if part:
            cleaned_parts.append(part)
    
    return cleaned_parts


# Apply the processing function and create new columns
new_cols = df['Tanı'].apply(process_tani).apply(pd.Series)

# Combine the new columns with the original dataframe
df = pd.concat([df.drop(columns=['Tanı']), new_cols], axis=1)

# Rename the new columns for clarity, starting from the next column index
for i in range(len(new_cols.columns)):
    df.rename(columns={i: f'Tanı_{i+1}'}, inplace=True)


In [ ]:
df['device'] = "compumedics"

display(df)

In [159]:
df.to_excel("/Users/bbkilboz/Desktop/export_compumedics.xlsx", index=False, header=True)
print(f"Data exported!")

Data exported!
